# 모델 배포 및 서빙

이 노트북은 학습된 머신러닝 모델을 API로 배포하고 서빙하는 방법을 보여줍니다.

## 학습 내용
- Flask를 사용한 REST API 생성
- FastAPI를 사용한 모던 API 생성
- 모델 예측 엔드포인트 구현
- API 테스트

In [ ]:
# 필요한 라이브러리 설치
!pip install -q flask fastapi uvicorn scikit-learn pandas numpy pydantic requests

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import joblib
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

## 1. 모델 학습 및 저장

In [ ]:
# 데이터 준비
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# 스케일러 학습
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 모델 및 스케일러 저장
joblib.dump(model, 'deployment_model.pkl')
joblib.dump(scaler, 'deployment_scaler.pkl')

print("모델과 스케일러가 저장되었습니다.")
print(f"학습 정확도: {model.score(X_train_scaled, y_train):.4f}")

## 2. Flask API 서버 코드 생성

In [ ]:
%%writefile flask_app.py
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# 모델 및 스케일러 로드
model = joblib.load('deployment_model.pkl')
scaler = joblib.load('deployment_scaler.pkl')

# 클래스 이름
class_names = ['setosa', 'versicolor', 'virginica']

@app.route('/')
def home():
    return jsonify({
        'message': 'Iris Classification API',
        'endpoints': {
            '/predict': 'POST - Make predictions',
            '/health': 'GET - Check API health'
        }
    })

@app.route('/health')
def health():
    return jsonify({'status': 'healthy'})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # 요청에서 데이터 가져오기
        data = request.get_json()
        
        # 특성 추출
        features = np.array(data['features']).reshape(1, -1)
        
        # 스케일링
        features_scaled = scaler.transform(features)
        
        # 예측
        prediction = model.predict(features_scaled)[0]
        probabilities = model.predict_proba(features_scaled)[0]
        
        # 결과 반환
        return jsonify({
            'prediction': class_names[prediction],
            'prediction_id': int(prediction),
            'probabilities': {
                class_names[i]: float(prob) 
                for i, prob in enumerate(probabilities)
            }
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)

## 3. FastAPI 서버 코드 생성

In [ ]:
%%writefile fastapi_app.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np
from typing import List

app = FastAPI(title="Iris Classification API", version="1.0")

# 모델 및 스케일러 로드
model = joblib.load('deployment_model.pkl')
scaler = joblib.load('deployment_scaler.pkl')

# 클래스 이름
class_names = ['setosa', 'versicolor', 'virginica']

# 요청 모델
class PredictionRequest(BaseModel):
    features: List[float]
    
    class Config:
        json_schema_extra = {
            "example": {
                "features": [5.1, 3.5, 1.4, 0.2]
            }
        }

# 응답 모델
class PredictionResponse(BaseModel):
    prediction: str
    prediction_id: int
    probabilities: dict

@app.get("/")
async def root():
    return {
        "message": "Iris Classification API",
        "version": "1.0",
        "docs": "/docs"
    }

@app.get("/health")
async def health():
    return {"status": "healthy"}

@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    try:
        # 특성 변환
        features = np.array(request.features).reshape(1, -1)
        
        # 입력 검증
        if features.shape[1] != 4:
            raise HTTPException(
                status_code=400, 
                detail="Expected 4 features"
            )
        
        # 스케일링
        features_scaled = scaler.transform(features)
        
        # 예측
        prediction = model.predict(features_scaled)[0]
        probabilities = model.predict_proba(features_scaled)[0]
        
        # 결과 반환
        return PredictionResponse(
            prediction=class_names[prediction],
            prediction_id=int(prediction),
            probabilities={
                class_names[i]: float(prob) 
                for i, prob in enumerate(probabilities)
            }
        )
    
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

## 4. API 서버 실행 (백그라운드)

**참고**: Colab에서는 백그라운드로 서버를 실행할 수 있습니다. 
로컬 환경에서는 터미널에서 직접 실행하세요.

Flask:
```bash
python flask_app.py
```

FastAPI:
```bash
uvicorn fastapi_app:app --host 0.0.0.0 --port 8000
```

In [ ]:
# Flask 서버를 백그라운드에서 실행
import subprocess
import time
import os
import signal

# 기존 프로세스 종료 (있는 경우)
try:
    !pkill -f flask_app.py
except:
    pass

# Flask 서버 시작
flask_process = subprocess.Popen(
    ['python', 'flask_app.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# 서버 시작 대기
time.sleep(3)
print(f"Flask 서버가 시작되었습니다 (PID: {flask_process.pid})")
print("서버 주소: http://localhost:5000")

## 5. API 테스트

In [ ]:
import requests
import json

# API 베이스 URL
BASE_URL = "http://localhost:5000"

# 헬스 체크
try:
    response = requests.get(f"{BASE_URL}/health")
    print("Health Check:")
    print(json.dumps(response.json(), indent=2))
    print()
except Exception as e:
    print(f"서버 연결 실패: {e}")
    print("서버가 실행 중인지 확인하세요.")

In [ ]:
# 예측 테스트 - Setosa
test_data_setosa = {
    "features": [5.1, 3.5, 1.4, 0.2]
}

try:
    response = requests.post(
        f"{BASE_URL}/predict",
        json=test_data_setosa
    )
    print("예측 결과 (Setosa):")
    print(json.dumps(response.json(), indent=2))
    print()
except Exception as e:
    print(f"예측 실패: {e}")

In [ ]:
# 예측 테스트 - Versicolor
test_data_versicolor = {
    "features": [6.3, 2.5, 4.9, 1.5]
}

try:
    response = requests.post(
        f"{BASE_URL}/predict",
        json=test_data_versicolor
    )
    print("예측 결과 (Versicolor):")
    print(json.dumps(response.json(), indent=2))
    print()
except Exception as e:
    print(f"예측 실패: {e}")

In [ ]:
# 예측 테스트 - Virginica
test_data_virginica = {
    "features": [7.7, 3.0, 6.1, 2.3]
}

try:
    response = requests.post(
        f"{BASE_URL}/predict",
        json=test_data_virginica
    )
    print("예측 결과 (Virginica):")
    print(json.dumps(response.json(), indent=2))
    print()
except Exception as e:
    print(f"예측 실패: {e}")

## 6. 배치 예측

In [ ]:
# 여러 샘플에 대한 배치 예측
test_samples = [
    {"name": "Sample 1", "features": [5.1, 3.5, 1.4, 0.2]},
    {"name": "Sample 2", "features": [6.3, 2.5, 4.9, 1.5]},
    {"name": "Sample 3", "features": [7.7, 3.0, 6.1, 2.3]}
]

results = []

for sample in test_samples:
    try:
        response = requests.post(
            f"{BASE_URL}/predict",
            json={"features": sample["features"]}
        )
        result = response.json()
        results.append({
            "name": sample["name"],
            "prediction": result["prediction"],
            "confidence": max(result["probabilities"].values())
        })
    except Exception as e:
        print(f"{sample['name']} 예측 실패: {e}")

# 결과를 데이터프레임으로 표시
if results:
    results_df = pd.DataFrame(results)
    print("\n배치 예측 결과:")
    print(results_df)

## 7. 서버 종료

In [ ]:
# Flask 서버 종료
try:
    flask_process.terminate()
    flask_process.wait(timeout=5)
    print("Flask 서버가 종료되었습니다.")
except Exception as e:
    print(f"서버 종료 중 오류: {e}")
    try:
        flask_process.kill()
        print("Flask 서버를 강제 종료했습니다.")
    except:
        pass

## 8. Docker 배포 예제

프로덕션 환경에서는 Docker를 사용하여 배포할 수 있습니다.

In [ ]:
%%writefile Dockerfile
FROM python:3.9-slim

WORKDIR /app

# 의존성 설치
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 애플리케이션 코드 복사
COPY fastapi_app.py .
COPY deployment_model.pkl .
COPY deployment_scaler.pkl .

# 포트 노출
EXPOSE 8000

# 서버 실행
CMD ["uvicorn", "fastapi_app:app", "--host", "0.0.0.0", "--port", "8000"]

In [ ]:
%%writefile requirements.txt
fastapi==0.104.1
uvicorn==0.24.0
scikit-learn==1.3.2
numpy==1.24.3
pandas==2.0.3
joblib==1.3.2
pydantic==2.5.0

Docker 이미지 빌드 및 실행:

```bash
# 이미지 빌드
docker build -t iris-api:v1 .

# 컨테이너 실행
docker run -d -p 8000:8000 --name iris-api iris-api:v1

# 로그 확인
docker logs iris-api

# 컨테이너 중지
docker stop iris-api
```

## 요약

이 노트북에서는 머신러닝 모델 배포에 대해 학습했습니다:
1. ✅ Flask를 사용한 간단한 REST API 구현
2. ✅ FastAPI를 사용한 모던 API 구현
3. ✅ 예측 엔드포인트 구현
4. ✅ API 테스트 및 배치 예측
5. ✅ Docker를 사용한 배포 설정

다음 단계: 모델 모니터링 및 성능 추적